In [22]:
from allensdk.api.queries.reference_space_api import ReferenceSpaceApi
import nibabel as nib
import nrrd
import numpy as np
import os
from pathlib import Path
ALLEN_RESOLUTIONS = [10, 25, 50, 100]


def download_allen_template(res=100, directory="."):
    """Downloading the Allen mouse brain template volume at a given resolution, and setting the correct affine
    Parameters
    ----------
    res : int
        Reference volume resolution. (Available resolutions : 10, 25, 50, 100 microns)
    directory : str
        Directory where the data will be saved.
    Returns
    -------
    nifti_file : str
        Absolute path to the downloaded template (nifti format, aligned to RAS+)
    """
    assert res in ALLEN_RESOLUTIONS, f"'res' must be in {ALLEN_RESOLUTIONS}"
    res = int(res)
    directory = Path(directory)
    directory.absolute()
    directory.mkdir(exist_ok=True, parents=True)

    # Preparing the filenames
    nrrd_file = directory / f"allen_template_{res}.nrrd"
    nifti_file = directory / f"allen_template_{res}.nii.gz"

    # Downloading the template
    rpa = ReferenceSpaceApi(base_uri=str(directory))
    rpa.download_template_volume(resolution=res, file_name=nrrd_file)

    # Loading the nrrd file
    vol, header = nrrd.read(nrrd_file)

    # Preparing the affine to align the template in the RAS+
    r_mm = res / 1e3
    affine = np.array([[ 0,      0,     r_mm, 0  ],
                       [ -r_mm,  0,     0,    0  ],
                       [ 0,     -r_mm,  0,    0  ],
                       [ 0,      0,     0,    1  ]])

    # Saving the volume (nifti)
    img = nib.Nifti1Image(vol, affine)

    #nib.save(img, nifti_file)
    nib.save(img, nifti_file)

    # Deleting the nrrd file
    os.remove(nrrd_file)

    return nifti_file

download_allen_template(res=100, directory='/Users/mahdi/stage-2022-mahdi/src')

2022-05-03 10:03:13,862 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://download.alleninstitute.org/informatics-archive/current-release/mouse_ccf/average_template/average_template_100.nrrd


PosixPath('/Users/mahdi/stage-2022-mahdi/src/allen_template_100.nii.gz')